# Linear Regression

In [1]:
import tensorflow as tf
print(tf.__version__)

2.0.0


In [2]:
# Disable eager execution
tf.compat.v1.disable_eager_execution()

In [2]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [3]:
# Clear any logs from previous runs
!rm -rf ./logs/ 

## Learning the TensorFlow way of linear regression

In [4]:
import numpy as np
import pandas as pd
import tensorflow_datasets as tfds
tfds.disable_progress_bar()

housing_url = 'http://archive.ics.uci.edu/ml/machine-learning-databases/housing/housing.data'
path = tf.keras.utils.get_file(housing_url.split("/")[-1], housing_url)

columns = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'MEDV']
data = pd.read_table(path, delim_whitespace=True, header=None, names=columns)

np.random.seed(1)
train = data.sample(frac=0.8).copy()
y_train = train['MEDV']
train.drop('MEDV', axis=1, inplace=True)

test = data.loc[~data.index.isin(train.index)].copy()
y_test = test['MEDV']
test.drop('MEDV', axis=1, inplace=True)

In [5]:
def define_feature_columns(data_df, categorical_cols, numeric_cols):
    feature_columns = []
    
    for feature_name in numeric_cols:
        feature_columns.append(tf.feature_column.numeric_column(feature_name, dtype=tf.float32))
    
    for feature_name in categorical_cols:
        vocabulary = data_df[feature_name].unique()
        feature_columns.append(tf.feature_column.categorical_column_with_vocabulary_list(feature_name, vocabulary))

    return feature_columns

In [6]:
def make_input_fn(data_df, label_df, num_epochs=10, shuffle=True, batch_size=256):
    
    def input_function():
        ds = tf.data.Dataset.from_tensor_slices((dict(data_df), label_df))
        if shuffle:
            ds = ds.shuffle(1000)
        ds = ds.batch(batch_size).repeat(num_epochs)
        return ds
    
    return input_function

In [7]:
categorical_cols = ['CHAS', 'RAD']
numeric_cols = ['CRIM', 'ZN', 'INDUS',  'NOX', 'RM', 'AGE', 'DIS', 'TAX', 'PTRATIO', 'B', 'LSTAT']
feature_columns = define_feature_columns(data, categorical_cols, numeric_cols)

train_input_fn = make_input_fn(train, y_train, num_epochs=1400)
test_input_fn = make_input_fn(test, y_test, num_epochs=1, shuffle=False)

In [8]:
output_dir = './logs/'
linear_est = tf.estimator.LinearRegressor(feature_columns=feature_columns,
                                          model_dir = output_dir,
                                          config=tf.estimator.RunConfig().replace(save_summary_steps=10))
linear_est.train(train_input_fn)
result = linear_est.evaluate(test_input_fn)

print(result)

INFO:tensorflow:Using config: {'_model_dir': './logs/', '_tf_random_seed': None, '_save_summary_steps': 10, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000000000F84BE88>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for up

NotFoundError: Failed to create a NewWriteableFile: ./logs/model.ckpt-0_temp_32134ed522e140a5bcd67b7843a8b290/part-00000-of-00001.data-00000-of-00001.tempstate15754753703399400385 : Impossibile trovare il percorso specificato.
; No such process
	 [[node save/SaveV2 (defined at C:\Users\Luca\anaconda3\lib\site-packages\tensorflow_core\python\framework\ops.py:1751) ]]

Original stack trace for 'save/SaveV2':
  File "C:\Users\Luca\anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\Luca\anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\Luca\anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\Luca\anaconda3\lib\site-packages\traitlets\config\application.py", line 664, in launch_instance
    app.start()
  File "C:\Users\Luca\anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 583, in start
    self.io_loop.start()
  File "C:\Us

In [ ]:
%tensorboard --logdir ./model_LinearRegressor
# http://localhost:6006/

### Interactions

In [7]:
def create_interactions(interactions_list, buckets=5):
    interactions = list()
    for (a, b) in interactions_list:
        interactions.append(tf.feature_column.crossed_column([a, b], hash_bucket_size=buckets))
    return interactions

derived_feature_columns = create_interactions([['RM', 'LSTAT']])
linear_est = tf.estimator.LinearRegressor(feature_columns=feature_columns+derived_feature_columns)
linear_est.train(train_input_fn)
result = linear_est.evaluate(test_input_fn)

print(result)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\LUCAMA~1.AZU\\AppData\\Local\\Temp\\tmppo3n7koe', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.

If you intended to run this layer in flo

In [8]:
def dicts_to_preds(pred_dicts):
    return np.array([pred['predictions'] for pred in pred_dicts])

preds = dicts_to_preds(linear_est.predict(test_input_fn))
print(preds)

INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\LUCAMA~1.AZU\AppData\Local\Temp\tmppo3n7koe\model.ckpt-2800
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
[[24.966026 ]
 [30.792831 ]
 [19.965616 ]
 [21.705784 ]
 [17.92263  ]
 [16.969158 ]
 [18.44628  ]
 [12.594122 ]
 [23.609722 ]
 [25.41783  ]
 [20.988722 ]
 [17.235489 ]
 [28.785534 ]
 [17.020382 ]
 [19.863

### Keras to estimator version

In [9]:
import tensorflow.keras as keras

In [10]:
def define_feature_columns_layers(data_df, categorical_cols, numeric_cols):
    feature_columns = []
    feature_layer_inputs = {}
    
    for feature_name in numeric_cols:
        feature_columns.append(tf.feature_column.numeric_column(feature_name, dtype=tf.float32))
        feature_layer_inputs[feature_name] = tf.keras.Input(shape=(1,), name=feature_name)
        
    for feature_name in categorical_cols:
        vocabulary = data_df[feature_name].unique()
        cat = tf.feature_column.categorical_column_with_vocabulary_list(feature_name, vocabulary)
        cat_one_hot = tf.feature_column.indicator_column(cat)
        feature_columns.append(cat_one_hot)
        feature_layer_inputs[feature_name] = tf.keras.Input(shape=(1,), name=feature_name, dtype=tf.int32)
    
    return feature_columns, feature_layer_inputs

In [11]:
def create_interactions(interactions_list, buckets=5):
    feature_columns = []
    
    for (a, b) in interactions_list:
        crossed_feature = tf.feature_column.crossed_column([a, b], hash_bucket_size=buckets)
        crossed_feature_one_hot = tf.feature_column.indicator_column(crossed_feature)
        feature_columns.append(crossed_feature_one_hot)
        
    return feature_columns

In [12]:
def create_linreg(feature_columns, feature_layer_inputs, optimizer):

    feature_layer = keras.layers.DenseFeatures(feature_columns)
    feature_layer_outputs = feature_layer(feature_layer_inputs)
    norm = keras.layers.BatchNormalization()(feature_layer_outputs)
    outputs = keras.layers.Dense(1, kernel_initializer='normal', activation='linear')(norm)
    
    model = keras.Model(inputs=[v for v in feature_layer_inputs.values()], outputs=outputs)
    model.compile(optimizer=optimizer, loss='mean_squared_error')
    return model

In [13]:
categorical_cols = ['CHAS', 'RAD']
numeric_cols = ['CRIM', 'ZN', 'INDUS',  'NOX', 'RM', 'AGE', 'DIS', 'TAX', 'PTRATIO', 'B', 'LSTAT']
feature_columns, feature_layer_inputs = define_feature_columns_layers(data, categorical_cols, numeric_cols)
interactions_columns = create_interactions([['RM', 'LSTAT']])

feature_columns += interactions_columns

optimizer = keras.optimizers.Ftrl(learning_rate=0.02)
model = create_linreg(feature_columns, feature_layer_inputs, optimizer)

Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.


In [14]:
import tempfile

def canned_keras(model):
    model_dir = tempfile.mkdtemp()
    keras_estimator = tf.keras.estimator.model_to_estimator(
        keras_model=model, model_dir=model_dir)
    return keras_estimator

estimator = canned_keras(model)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using the Keras model provided.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\LUCAMA~1.AZU\\AppData\\Local\\Temp\\tmpv_bcjh72', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [15]:
train_input_fn = make_input_fn(train, y_train, num_epochs=1400)
test_input_fn = make_input_fn(test, y_test, num_epochs=1, shuffle=False)

estimator.train(train_input_fn)
result = estimator.evaluate(test_input_fn)

print(result)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Warm-starting with WarmStartSettings: WarmStartSettings(ckpt_to_initialize_from='C:\\Users\\LUCAMA~1.AZU\\AppData\\Local\\Temp\\tmpv_bcjh72\\keras\\keras_model.ckpt', vars_to_warm_start='.*', var_name_to_vocab_info={}, var_name_to_prev_var_name={})
INFO:tensorflow:Warm-starting from: C:\Users\LUCAMA~1.AZU\AppData\Local\Temp\tmpv_bcjh72\keras\keras_model.ckpt
INFO:tensorflow:Warm-starting variables only in TRAINABLE_VARIABLES.
INFO:tensorflow:Warm-started 4 variables.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into C:\Users\LUCAMA~1.AZU\AppData\Local\Temp\tmpv_bcjh72\model.ckpt.
INFO:tensorflow:loss = 605.7178, step = 0
INFO:tensorflow:global_step/sec: 195.312
INFO:tensorflow:loss = 300.6946, step = 100 (0.514 sec)
INFO:tensorflow:global_st

## Understanding coefficients in linear regression

In [16]:
weights = estimator.get_variable_value('layer_with_weights-1/kernel/.ATTRIBUTES/VARIABLE_VALUE')

In [17]:
print(weights)

[[ 0.01552782]
 [ 0.07513991]
 [-0.05228834]
 [-0.0862719 ]
 [-0.01169319]
 [ 0.00709739]
 [ 0.08730743]
 [-0.00230968]
 [ 0.02895174]
 [ 0.06118769]
 [ 0.05902488]
 [-0.13423888]
 [-0.02005083]
 [ 0.01062155]
 [ 0.02191463]
 [ 0.05915826]
 [-0.03479118]
 [ 0.05837935]
 [ 0.04497513]
 [-0.00108679]
 [ 0.15156531]
 [-0.01708724]
 [-0.06367704]
 [ 0.00842286]
 [ 0.0287493 ]
 [ 0.06794064]
 [ 0.02151924]]


In [18]:
def extract_labels(feature_columns):
    labels = list()
    for col in feature_columns:
        col_config = col.get_config()
        if 'key' in col_config:
            labels.append(col_config['key'])
        elif 'categorical_column' in col_config:
            if col_config['categorical_column']['class_name']=='VocabularyListCategoricalColumn':
                key = col_config['categorical_column']['config']['key']
                for item in col_config['categorical_column']['config']['vocabulary_list']:
                     labels.append(key+'_val='+str(item))
            elif col_config['categorical_column']['class_name']=='CrossedColumn':
                keys = col_config['categorical_column']['config']['keys']
                for bucket in range(col_config['categorical_column']['config']['hash_bucket_size']):
                    labels.append('x'.join(keys)+'_bkt_'+str(bucket))
    return labels

In [20]:
labels = extract_labels(feature_columns)

for label, weight in zip(labels, weights):
    print(f"{label:15s} : {weight[0]:+.2f}")

CRIM            : +0.02
ZN              : +0.08
INDUS           : -0.05
NOX             : -0.09
RM              : -0.01
AGE             : +0.01
DIS             : +0.09
TAX             : -0.00
PTRATIO         : +0.03
B               : +0.06
LSTAT           : +0.06
CHAS_val=0      : -0.13
CHAS_val=1      : -0.02
RAD_val=1       : +0.01
RAD_val=2       : +0.02
RAD_val=3       : +0.06
RAD_val=5       : -0.03
RAD_val=4       : +0.06
RAD_val=8       : +0.04
RAD_val=6       : -0.00
RAD_val=7       : +0.15
RAD_val=24      : -0.02
RMxLSTAT_bkt_0  : -0.06
RMxLSTAT_bkt_1  : +0.01
RMxLSTAT_bkt_2  : +0.03
RMxLSTAT_bkt_3  : +0.07
RMxLSTAT_bkt_4  : +0.02


## Understanding loss functions in linear regression

In [21]:
def create_linreg(feature_columns, feature_layer_inputs, optimizer, 
                  loss='mean_squared_error', metrics=['mean_absolute_error']):

    feature_layer = keras.layers.DenseFeatures(feature_columns)
    feature_layer_outputs = feature_layer(feature_layer_inputs)
    norm = keras.layers.BatchNormalization()(feature_layer_outputs)
    outputs = keras.layers.Dense(1, kernel_initializer='normal', activation='linear')(norm)
    
    model = keras.Model(inputs=[v for v in feature_layer_inputs.values()], outputs=outputs)
    model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
    return model

In [22]:
categorical_cols = ['CHAS', 'RAD']
numeric_cols = ['CRIM', 'ZN', 'INDUS',  'NOX', 'RM', 'AGE', 'DIS', 'TAX', 'PTRATIO', 'B', 'LSTAT']
feature_columns, feature_layer_inputs = define_feature_columns_layers(data, categorical_cols, numeric_cols)
interactions_columns = create_interactions([['RM', 'LSTAT']])

feature_columns += interactions_columns

optimizer = keras.optimizers.Ftrl(learning_rate=0.02)
model = create_linreg(feature_columns, feature_layer_inputs, optimizer,
                      loss='mean_absolute_error', 
                      metrics=['mean_absolute_error', 'mean_squared_error'])

estimator = canned_keras(model)

train_input_fn = make_input_fn(train, y_train, num_epochs=1400)
test_input_fn = make_input_fn(test, y_test, num_epochs=1, shuffle=False)

estimator.train(train_input_fn)
result = estimator.evaluate(test_input_fn)

print(result)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using the Keras model provided.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\LUCAMA~1.AZU\\AppData\\Local\\Temp\\tmpsovirc_0', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling mod

## Implementing lasso and ridge regression

In [23]:
def create_ridge_linreg(feature_columns, feature_layer_inputs, optimizer, 
                        loss='mean_squared_error', metrics=['mean_absolute_error'],
                        l2=0.01):
    
    regularizer = keras.regularizers.l2(l2)

    feature_layer = keras.layers.DenseFeatures(feature_columns)
    feature_layer_outputs = feature_layer(feature_layer_inputs)
    norm = keras.layers.BatchNormalization()(feature_layer_outputs)
    outputs = keras.layers.Dense(1, 
                                 kernel_initializer='normal', 
                                 kernel_regularizer = regularizer, 
                                 activation='linear')(norm)
    
    model = keras.Model(inputs=[v for v in feature_layer_inputs.values()], outputs=outputs)
    model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
    return model

In [24]:
categorical_cols = ['CHAS', 'RAD']
numeric_cols = ['CRIM', 'ZN', 'INDUS',  'NOX', 'RM', 'AGE', 'DIS', 'TAX', 'PTRATIO', 'B', 'LSTAT']
feature_columns, feature_layer_inputs = define_feature_columns_layers(data, categorical_cols, numeric_cols)
interactions_columns = create_interactions([['RM', 'LSTAT']])

feature_columns += interactions_columns

optimizer = keras.optimizers.Ftrl(learning_rate=0.02)
model = create_ridge_linreg(feature_columns, feature_layer_inputs, optimizer,
                      loss='mean_squared_error', 
                      metrics=['mean_absolute_error', 'mean_squared_error'],
                           l2=0.01)

estimator = canned_keras(model)

train_input_fn = make_input_fn(train, y_train, num_epochs=1400)
test_input_fn = make_input_fn(test, y_test, num_epochs=1, shuffle=False)

estimator.train(train_input_fn)
result = estimator.evaluate(test_input_fn)

print(result)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using the Keras model provided.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\LUCAMA~1.AZU\\AppData\\Local\\Temp\\tmpznihab50', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling mod

In [25]:
def create_lasso_linreg(feature_columns, feature_layer_inputs, optimizer, 
                        loss='mean_squared_error', metrics=['mean_absolute_error'],
                        l1=0.001):
    
    regularizer = keras.regularizers.l1(l1)

    feature_layer = keras.layers.DenseFeatures(feature_columns)
    feature_layer_outputs = feature_layer(feature_layer_inputs)
    norm = keras.layers.BatchNormalization()(feature_layer_outputs)
    outputs = keras.layers.Dense(1, 
                                 kernel_initializer='normal', 
                                 kernel_regularizer = regularizer, 
                                 activation='linear')(norm)
    
    model = keras.Model(inputs=[v for v in feature_layer_inputs.values()], outputs=outputs)
    model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
    return model

In [26]:
categorical_cols = ['CHAS', 'RAD']
numeric_cols = ['CRIM', 'ZN', 'INDUS',  'NOX', 'RM', 'AGE', 'DIS', 'TAX', 'PTRATIO', 'B', 'LSTAT']
feature_columns, feature_layer_inputs = define_feature_columns_layers(data, categorical_cols, numeric_cols)
interactions_columns = create_interactions([['RM', 'LSTAT']])

feature_columns += interactions_columns

optimizer = keras.optimizers.Ftrl(learning_rate=0.02)
model = create_lasso_linreg(feature_columns, feature_layer_inputs, optimizer,
                      loss='mean_squared_error', 
                      metrics=['mean_absolute_error', 'mean_squared_error'],
                           l1=0.001)

estimator = canned_keras(model)

train_input_fn = make_input_fn(train, y_train, num_epochs=1400)
test_input_fn = make_input_fn(test, y_test, num_epochs=1, shuffle=False)

estimator.train(train_input_fn)
result = estimator.evaluate(test_input_fn)

print(result)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using the Keras model provided.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\LUCAMA~1.AZU\\AppData\\Local\\Temp\\tmp9j35shbg', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling mod

## Implementing elastic net regression

In [27]:
def create_elasticnet_linreg(feature_columns, feature_layer_inputs, optimizer, 
                        loss='mean_squared_error', metrics=['mean_absolute_error'],
                        l1=0.001, l2=0.01):
    
    regularizer = keras.regularizers.l1_l2(l1=l1, l2=l2)

    feature_layer = keras.layers.DenseFeatures(feature_columns)
    feature_layer_outputs = feature_layer(feature_layer_inputs)
    norm = keras.layers.BatchNormalization()(feature_layer_outputs)
    outputs = keras.layers.Dense(1, 
                                 kernel_initializer='normal', 
                                 kernel_regularizer = regularizer, 
                                 activation='linear')(norm)
    
    model = keras.Model(inputs=[v for v in feature_layer_inputs.values()], outputs=outputs)
    model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
    return model

In [28]:
categorical_cols = ['CHAS', 'RAD']
numeric_cols = ['CRIM', 'ZN', 'INDUS',  'NOX', 'RM', 'AGE', 'DIS', 'TAX', 'PTRATIO', 'B', 'LSTAT']
feature_columns, feature_layer_inputs = define_feature_columns_layers(data, categorical_cols, numeric_cols)
interactions_columns = create_interactions([['RM', 'LSTAT']])

feature_columns += interactions_columns

optimizer = keras.optimizers.Ftrl(learning_rate=0.02)
model = create_elasticnet_linreg(feature_columns, feature_layer_inputs, optimizer,
                      loss='mean_squared_error', 
                      metrics=['mean_absolute_error', 'mean_squared_error'],
                           l1=0.001, l2=0.01)

estimator = canned_keras(model)

train_input_fn = make_input_fn(train, y_train, num_epochs=1400)
test_input_fn = make_input_fn(test, y_test, num_epochs=1, shuffle=False)

estimator.train(train_input_fn)
result = estimator.evaluate(test_input_fn)

print(result)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using the Keras model provided.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\LUCAMA~1.AZU\\AppData\\Local\\Temp\\tmp45t6ixjc', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling mod

## Implementing logistic regression

In [29]:
breast_cancer = 'https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/breast-cancer-wisconsin.data'
path = tf.keras.utils.get_file(breast_cancer.split("/")[-1], breast_cancer)

columns = ['sample_code', 'clump_thickness', 'cell_size_uniformity', 'cell_shape_uniformity',
           'marginal_adhesion', 'single_epithelial_cell_size', 'bare_nuclei', 'bland_chromatin',
           'normal_nucleoli', 'mitoses', 'class']

data = pd.read_csv(path, header=None, names=columns, na_values=[np.nan, '?'])
data = data.fillna(data.median())

np.random.seed(1)
train = data.sample(frac=0.8).copy()
y_train = (train['class']==4).astype(int)
train.drop(['sample_code', 'class'], axis=1, inplace=True)

test = data.loc[~data.index.isin(train.index)].copy()
y_test = (test['class']==4).astype(int)
test.drop(['sample_code', 'class'], axis=1, inplace=True)

In [30]:
def create_logreg(feature_columns, feature_layer_inputs, optimizer, 
                  loss='binary_crossentropy', metrics=['accuracy'],
                  l2=0.01):
    
    regularizer = keras.regularizers.l2(l2)

    feature_layer = keras.layers.DenseFeatures(feature_columns)
    feature_layer_outputs = feature_layer(feature_layer_inputs)
    norm = keras.layers.BatchNormalization()(feature_layer_outputs)
    outputs = keras.layers.Dense(1, 
                                 kernel_initializer='normal', 
                                 kernel_regularizer = regularizer, 
                                 activation='sigmoid')(norm)
    
    model = keras.Model(inputs=[v for v in feature_layer_inputs.values()], outputs=outputs)
    model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
    return model

In [31]:
categorical_cols = []
numeric_cols = ['clump_thickness', 'cell_size_uniformity', 'cell_shape_uniformity',
                'marginal_adhesion', 'single_epithelial_cell_size', 'bare_nuclei', 'bland_chromatin',
                'normal_nucleoli', 'mitoses']

feature_columns, feature_layer_inputs = define_feature_columns_layers(data, categorical_cols, numeric_cols)

optimizer = keras.optimizers.Ftrl(learning_rate=0.02)
model = create_logreg(feature_columns, feature_layer_inputs, optimizer, l2=0.01)

estimator = canned_keras(model)

train_input_fn = make_input_fn(train, y_train, num_epochs=300, batch_size=32)
test_input_fn = make_input_fn(test, y_test, num_epochs=1, shuffle=False)

estimator.train(train_input_fn)
result = estimator.evaluate(test_input_fn)

print(result)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using the Keras model provided.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\LUCAMA~1.AZU\\AppData\\Local\\Temp\\tmp3g1fr9sj', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling mod

## Resorting to non-linear regression

In [32]:
try:
    from tensorflow.python.keras.layers.kernelized import RandomFourierFeatures
except:
    # from TF 2.2
    from tensorflow.keras.layers.experimental import RandomFourierFeatures 

In [33]:
def create_svc(feature_columns, feature_layer_inputs, optimizer, 
               loss='hinge', metrics=['accuracy'],
               l2=0.01, output_dim=64, scale=None):
    
    regularizer = keras.regularizers.l2(l2)

    feature_layer = keras.layers.DenseFeatures(feature_columns)
    feature_layer_outputs = feature_layer(feature_layer_inputs)
    norm = keras.layers.BatchNormalization()(feature_layer_outputs)
    rff = RandomFourierFeatures(output_dim=output_dim, scale=scale, kernel_initializer='gaussian')(norm)
    outputs = keras.layers.Dense(1, 
                                 kernel_initializer='normal', 
                                 kernel_regularizer = regularizer, 
                                 activation='sigmoid')(rff)
    
    model = keras.Model(inputs=[v for v in feature_layer_inputs.values()], outputs=outputs)
    model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
    return model

In [34]:
categorical_cols = []
numeric_cols = ['clump_thickness', 'cell_size_uniformity', 'cell_shape_uniformity',
                'marginal_adhesion', 'single_epithelial_cell_size', 'bare_nuclei', 'bland_chromatin',
                'normal_nucleoli', 'mitoses']

feature_columns, feature_layer_inputs = define_feature_columns_layers(data, categorical_cols, numeric_cols)

optimizer = keras.optimizers.Adam(learning_rate=0.01)
model = create_svc(feature_columns, feature_layer_inputs, optimizer, 
                   loss='hinge', l2=0.001, output_dim=512)

estimator = canned_keras(model)

train_input_fn = make_input_fn(train, y_train, num_epochs=300, batch_size=32)
test_input_fn = make_input_fn(test, y_test, num_epochs=1, shuffle=False)

estimator.train(train_input_fn)
result = estimator.evaluate(test_input_fn)

print(result)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using the Keras model provided.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\LUCAMA~1.AZU\\AppData\\Local\\Temp\\tmprrf7djzy', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_ev

INFO:tensorflow:Saving 'checkpoint_path' summary for global step 5400: C:\Users\LUCAMA~1.AZU\AppData\Local\Temp\tmprrf7djzy\model.ckpt-5400
{'accuracy': 0.9357143, 'loss': 0.71555066, 'global_step': 5400}


## Preparing Wide & Deep Learning for Recommender Systems 

In [35]:
census_dir = 'https://archive.ics.uci.edu/ml/machine-learning-databases/adult/'
train_path = tf.keras.utils.get_file('adult.data', census_dir + 'adult.data')
test_path = tf.keras.utils.get_file('adult.test', census_dir + 'adult.test')

columns = ['age', 'workclass', 'fnlwgt', 'education', 'education_num',
           'marital_status', 'occupation', 'relationship', 'race', 'gender',
           'capital_gain', 'capital_loss', 'hours_per_week', 'native_country',
           'income_bracket']

train_data = pd.read_csv(train_path, header=None, names=columns)
test_data = pd.read_csv(test_path, header=None, names=columns, skiprows=1)

In [36]:
predictors = ['age', 'workclass', 'education', 'education_num',
              'marital_status', 'occupation', 'relationship', 'gender']

y_train = (train_data.income_bracket==' >50K').astype(int)
y_test = (test_data.income_bracket==' >50K.').astype(int)

train_data = train_data[predictors]
test_data = test_data[predictors]

In [37]:
train_data[['age', 'education_num']] = train_data[['age', 'education_num']].fillna(train_data[['age', 'education_num']].mean())
test_data[['age', 'education_num']] = test_data[['age', 'education_num']].fillna(train_data[['age', 'education_num']].mean())

In [38]:
def define_feature_columns(data_df, numeric_cols, categorical_cols, categorical_embeds, dimension=30):
    numeric_columns = []
    categorical_columns = []
    embeddings = []
    
    for feature_name in numeric_cols:
        numeric_columns.append(tf.feature_column.numeric_column(feature_name, dtype=tf.float32))
    
    for feature_name in categorical_cols:
        vocabulary = data_df[feature_name].unique()
        categorical_columns.append(tf.feature_column.categorical_column_with_vocabulary_list(feature_name, vocabulary))
        
    for feature_name in categorical_embeds:
        vocabulary = data_df[feature_name].unique()
        to_categorical = tf.feature_column.categorical_column_with_vocabulary_list(feature_name, vocabulary)
        embeddings.append(tf.feature_column.embedding_column(to_categorical, dimension=dimension))
        
    return numeric_columns, categorical_columns, embeddings

def create_interactions(interactions_list, buckets=10):
    feature_columns = []
    
    for (a, b) in interactions_list:
        crossed_feature = tf.feature_column.crossed_column([a, b], hash_bucket_size=buckets)
        crossed_feature_one_hot = tf.feature_column.indicator_column(crossed_feature)
        feature_columns.append(crossed_feature_one_hot)
        
    return feature_columns

In [39]:
numeric_columns, categorical_columns, embeddings = define_feature_columns(train_data, 
                                                                          numeric_cols=['age', 'education_num'], 
                                                                          categorical_cols=['gender'], 
                                                                          categorical_embeds=['workclass', 'education',
                                                                                              'marital_status', 'occupation', 
                                                                                              'relationship'], 
                                                                          dimension=32)

interactions = create_interactions([['education', 'occupation']], buckets=10)

In [40]:
estimator = tf.estimator.DNNLinearCombinedClassifier(
    # wide settings
    linear_feature_columns=numeric_columns+categorical_columns+interactions,
    linear_optimizer=keras.optimizers.Ftrl(learning_rate=0.02),
    # deep settings
    dnn_feature_columns=embeddings,
    dnn_hidden_units=[1024, 512, 128, 64],
    dnn_optimizer=keras.optimizers.Adam(learning_rate=0.01))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\LUCAMA~1.AZU\\AppData\\Local\\Temp\\tmp25oxd1at', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [41]:
def make_input_fn(data_df, label_df, num_epochs=10, shuffle=True, batch_size=256):
    
    def input_function():
        ds = tf.data.Dataset.from_tensor_slices((dict(data_df), label_df))
        if shuffle:
            ds = ds.shuffle(1000)
        ds = ds.batch(batch_size).repeat(num_epochs)
        return ds
    
    return input_function

In [42]:
train_input_fn = make_input_fn(train_data, y_train, num_epochs=500, batch_size=256)
test_input_fn = make_input_fn(test_data, y_test, num_epochs=1, shuffle=False)

In [43]:
estimator.train(input_fn=train_input_fn, steps=1000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into C:\Users\LUCAMA~1.AZU\AppData\Local\Temp\tmp25oxd1at\model.ckpt.
INFO:tensorflow:loss = 0.6887884, step = 0
INFO:tensorflow:global_step/sec: 57.1429
INFO:tensorflow:loss = 0.37364495, step = 100 (1.752 sec)
INFO:tensorflow:global_step/sec: 74.8502
INFO:tensorflow:loss = 0.40053362, step = 200 (1.336 sec)
INFO:tensorflow:global_step/sec: 74.6268
INFO:tensorflow:loss = 0.40525287, step = 300 (1.340 sec)
INFO:tensorflow:global_step/sec: 75.0752
INFO:tensorflow:loss = 0.28332102, step = 400 (1.333 sec)
INFO:tensorflow:global_step/sec: 74.9625
INFO:tensorflow:loss = 0.34983099, step = 500 (1.333 sec)
INFO:tensorflow:global_step/sec: 74.7943
INFO:tensorflow:loss = 0.38028675, step = 600 (1.337 sec)
INFO:tensorflow

In [44]:
results = estimator.evaluate(input_fn=test_input_fn)
print(results)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2020-05-11T15:53:21Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\LUCAMA~1.AZU\AppData\Local\Temp\tmp25oxd1at\model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 2.24300s
INFO:tensorflow:Finished evaluation at 2020-05-11-15:53:23
INFO:tensorflow:Saving dict for global step 1000: accuracy = 0.83391684, accuracy_baseline = 0.76377374, auc = 0.88012385, auc_precision_recall = 0.68032277, average_loss = 0.35969484, global_step = 1000, label/mean = 0.23622628, loss = 0.35985297, precision = 0.70583993, prediction/mean = 0.21803579, recall = 0.5091004
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 1000: C:\Users\LUCAMA~1.AZU\AppData\Local\Temp\tmp25oxd1at\model.ckpt-1000
{'accuracy': 0.83391684, 'accuracy_baseline': 0.76377374, 'auc': 0.880123

In [45]:
def predict_proba(predictor):
    preds = list()
    for pred in predictor:
        preds.append(pred['probabilities'])
    return np.array(preds)

predictions = predict_proba(estimator.predict(input_fn=test_input_fn))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\LUCAMA~1.AZU\AppData\Local\Temp\tmp25oxd1at\model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [46]:
predictions

array([[9.9922061e-01, 7.7938952e-04],
       [8.7542838e-01, 1.2457163e-01],
       [3.6073923e-01, 6.3926083e-01],
       ...,
       [2.7226123e-01, 7.2773874e-01],
       [9.3168575e-01, 6.8314269e-02],
       [3.0533168e-01, 6.9466835e-01]], dtype=float32)